# OpenCVとDlibを用いた画像認識 ｰ深層学習ｰ

ここでは、カメラから取得した映像を用いて画像認識を行い、
必要な情報を取得するための流れを学ぶことで、
画像認識をビジネス現場で応用するイメージをつかみます。

### 【上級】MobileNetSSDを用いた人の検出

OpenCVとMobileNet SSDモデルを使用して、ビデオ内の人物を検出するプログラムです。主な機能は以下の通りです：

1. 事前学習済みのMobileNet SSDモデルを読み込みます。
2. 指定されたビデオファイルを開き、フレームごとに処理を行います。
3. 各フレームに対して、MobileNet SSDモデルを使用して物体検出を実行します。
4. 検出された物体の中から「人」のみを抽出し、バウンディングボックスと信頼度を表示します。
5. 処理結果をリアルタイムで画面に表示します。
6. Qキーが押されるまで処理を続け、終了時にはリソースを解放します。

MobileNetSSDのは、次のページをしてください。

[MobileNetSSD](Dhttps://github.com/chuanqi305/MobileNet-SSD/tree/master)

In [1]:
import cv2
import numpy as np

# MobileNet SSDモデルの設定ファイルのパスを指定
prototxt_path = 'det/MobileNetSSD_deploy.prototxt'
model_path = 'det/MobileNetSSD_deploy.caffemodel'

# MobileNet SSDが検出できるオブジェクトクラスのリスト
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

# 事前学習済みのMobileNet SSDモデルを読み込む
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# ビデオファイルを開く
cap = cv2.VideoCapture('vtest.avi')

# メインループ：フレームごとに処理を行う
while True:
    # ビデオからフレームを1枚読み込む
    ret, frame = cap.read()
    if not ret:
        break  # ビデオの終わりに達したらループを抜ける

    # フレームの高さと幅を取得
    (h, w) = frame.shape[:2]

    # 入力画像を前処理し、ニューラルネットワークに入力できる形式に変換
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)),
                                 0.007843, (300, 300), 127.5)

    # ニューラルネットワークに入力データをセットし、順伝播を実行
    net.setInput(blob)
    detections = net.forward()

    # 検出結果をループで処理
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        # 信頼度が0.2より高い検出結果のみを処理
        if confidence > 0.2:
            idx = int(detections[0, 0, i, 1])
            # 検出されたオブジェクトが「人」の場合のみ処理
            if CLASSES[idx] == "person":
                # バウンディングボックスの座標を計算
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # フレーム上にバウンディングボックスとラベルを描画
                label = f"Person: {confidence:.2f}"
                cv2.rectangle(frame, (startX, startY), (endX, endY),
                              (0, 255, 0), 2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(frame, label, (startX, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 処理結果のフレームを表示
    cv2.imshow("Person Detection", frame)

    # キー入力を1ミリ秒待機し、'q'キーが押されたかチェック
    # 0xFF == ord('q')は、押されたキーが'q'かどうかを確認
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # 'q'キーが押されたらループを終了
        

# リソースを解放
cap.release()
cv2.destroyAllWindows()

### 【上級】MobileNetSSDの検出結果を用いた人追跡

このPythonスクリプトは、リアルタイムの人物追跡システムを実装しています。主な機能は以下の通りです：

1. OpenCVとMobileNet SSDモデルを使用して、カメラフィードから人物を検出します。
2. 検出された人物に対して個別のトラッカーを割り当て、フレーム間で追跡を行います。
3. 一定間隔で再検出を行い、新しい人物の出現や既存の追跡対象の位置を更新します。
4. 各追跡対象に一意のIDと色を割り当て、バウンディングボックスと移動軌跡を表示します。
5. データアソシエーションを使用して、検出結果と既存のトラッカーをマッチングします。
データアソシエーションとは、複数のフレームや時間にわたって検出された物体を、同一の物体として関連付ける処理のことです。
6. 追跡に失敗したオブジェクトを適切に管理し、システムの安定性を保ちます。

In [3]:
import cv2  
import numpy as np  

# 事前学習済みのMobileNet SSDモデルを人検出のためにロードします
prototxt_path = 'det/MobileNetSSD_deploy.prototxt'  # モデルの構造を定義するプロトテキストファイル
model_path = 'det/MobileNetSSD_deploy.caffemodel'   # 学習済みの重みが保存されたモデルファイル

# MobileNet SSDが検出可能なクラスラベルのリストを初期化します
# このリストには様々なオブジェクトが含まれていますが、このプログラムでは主に"person"を使用します
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

# OpenCVの深層学習モジュールを使用してモデルを読み込みます
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# カメラからのビデオキャプチャを初期化します
# 0はデフォルトのカメラを指し、cv2.CAP_DSHOWはDirectShowを使用することを指定します
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# 追跡対象の管理に使用する変数を初期化します
trackers = {}      # 各追跡対象のトラッカーを格納する辞書
track_id = 0       # 追跡対象を一意に識別するためのID
colors = {}        # 各追跡対象に割り当てる色を格納する辞書
trajectories = {}  # 各追跡対象の移動軌跡を格納する辞書

# フレームカウントと再検出の間隔を設定します
frame_count = 0
RE_DETECT_INTERVAL = 10  # 10フレームごとに再検出を行います

# IoU（Intersection over Union）を計算する関数を定義します
# IoUは2つのバウンディングボックスがどれだけ重なっているかを示す指標です
def compute_iou(box1, box2):
    """2つのバウンディングボックスのIoUを計算します。"""
    x1, y1, w1, h1 = box1
    x1_max = x1 + w1
    y1_max = y1 + h1

    x2, y2, w2, h2 = box2
    x2_max = x2 + w2
    y2_max = y2 + h2

    # 重なり部分の座標を計算します
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1_max, x2_max)
    yi2 = min(y1_max, y2_max)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    # 各ボックスの面積を計算します
    box1_area = w1 * h1
    box2_area = w2 * h2

    # IoUを計算します
    union_area = box1_area + box2_area - inter_area

    if union_area == 0:
        return 0.0  # ゼロ除算を避けるため

    iou = inter_area / union_area
    return iou

# メインのループ処理を開始します
while True:
    # 新しいフレームを読み込みます
    ret, frame = cap.read()
    if not ret:
        break  # フレームが取得できなければループを抜けます

    frame_count += 1  # フレームカウンタを増加させます

    # フレームのサイズを取得します
    (h, w) = frame.shape[:2]

    # 一定間隔（RE_DETECT_INTERVAL）で再検出を行います
    if frame_count % RE_DETECT_INTERVAL == 0:
        # フレームを検出用に前処理します
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)),
                                     0.007843, (300, 300), 127.5)

        # モデルに入力を設定し、推論を行います
        net.setInput(blob)
        detections = net.forward()

        # 検出結果を格納するリストを初期化します
        detections_list = []

        # 検出結果をループ処理します
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            # 信頼度が0.5以上の検出結果のみを処理します
            if confidence > 0.5:
                idx = int(detections[0, 0, i, 1])

                # クラスが「person」の場合のみ処理します
                if CLASSES[idx] == "person":
                    # バウンディングボックスの座標を計算します
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")

                    # バウンディングボックスを（x, y, w, h）の形式に変換します
                    bbox = (startX, startY, endX - startX, endY - startY)
                    detections_list.append(bbox)

        # データアソシエーション：検出結果と既存のトラッカーをマッチングします
        unmatched_detections = detections_list.copy()
        unmatched_trackers = list(trackers.keys())

        # 各検出結果に対して、既存のトラッカーとマッチングを試みます
        for det in detections_list:
            best_iou = 0
            best_tracker_id = None
            for trk_id in unmatched_trackers:
                # トラッカーの最後のバウンディングボックスを取得します
                tracker = trackers[trk_id]['tracker']
                trk_bbox = trackers[trk_id]['bbox']

                # IoUを計算します
                iou = compute_iou(det, trk_bbox)

                # 最もIoUが高いトラッカーを見つけます
                if iou > best_iou:
                    best_iou = iou
                    best_tracker_id = trk_id

            # IoUが閾値を超える場合、トラッカーを更新します
            if best_iou > 0.3:
                # トラッカーを新しい検出結果で初期化します
                tracker = trackers[best_tracker_id]['tracker']
                ok = tracker.init(frame, det)
                if ok:
                    trackers[best_tracker_id]['bbox'] = det
                    # 軌跡に現在の位置を追加します
                    trackers[best_tracker_id]['trajectory'].append(
                        (int(det[0] + det[2] / 2), int(det[1] + det[3] / 2)))
                    unmatched_detections.remove(det)
                    unmatched_trackers.remove(best_tracker_id)
                else:
                    # トラッカーの再初期化が失敗した場合、新しいトラッカーを作成します
                    try:
                        tracker = cv2.TrackerCSRT_create()
                    except AttributeError:
                        tracker = cv2.legacy.TrackerCSRT_create()
                    tracker.init(frame, det)
                    trackers[best_tracker_id]['tracker'] = tracker
                    trackers[best_tracker_id]['bbox'] = det
                    trackers[best_tracker_id]['trajectory'] = [
                        (int(det[0] + det[2] / 2), int(det[1] + det[3] / 2))]
                    unmatched_detections.remove(det)
                    unmatched_trackers.remove(best_tracker_id)

        # マッチしなかった検出結果について、新しいトラッカーを作成します
        for det in unmatched_detections:
            # 新しいトラッカーを作成します
            try:
                tracker = cv2.TrackerCSRT_create()
            except AttributeError:
                tracker = cv2.legacy.TrackerCSRT_create()
            ok = tracker.init(frame, det)
            if ok:
                trackers[track_id] = {
                    'tracker': tracker,
                    'bbox': det,
                    'trajectory': [(int(det[0] + det[2] / 2), int(det[1] + det[3] / 2))]
                }
                # ランダムな色を生成してトラッカーに割り当てます
                colors[track_id] = (int(np.random.randint(0, 255)),
                                    int(np.random.randint(0, 255)),
                                    int(np.random.randint(0, 255)))
                track_id += 1  # トラックIDを増加させます

        # マッチしなかったトラッカーを削除します
        for trk_id in unmatched_trackers:
            del trackers[trk_id]
            del colors[trk_id]

    else:
        # 既存のトラッカーを更新します
        to_delete = []
        for trk_id in list(trackers.keys()):
            tracker = trackers[trk_id]['tracker']
            ok, bbox = tracker.update(frame)
            if ok:
                # トラッキングが成功した場合、バウンディングボックスを更新します
                trackers[trk_id]['bbox'] = bbox
                (x, y, w, h) = [int(v) for v in bbox]
                # バウンディングボックスを描画します
                cv2.rectangle(frame, (x, y), (x + w, y + h), colors[trk_id], 2)

                # トラックIDを表示します
                cv2.putText(frame, f'ID {trk_id}', (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, colors[trk_id], 2)

                # 軌跡を更新します
                center = (int(x + w / 2), int(y + h / 2))
                trackers[trk_id]['trajectory'].append(center)

                # 軌跡を描画します
                trajectory = trackers[trk_id]['trajectory']
                for i in range(1, len(trajectory)):
                    cv2.line(frame, trajectory[i - 1], trajectory[i], colors[trk_id], 2)
            else:
                # トラッキングが失敗した場合、削除リストに追加します
                to_delete.append(trk_id)

        # 失敗したトラッカーを削除します
        for trk_id in to_delete:
            del trackers[trk_id]
            del colors[trk_id]

    # フレームを表示します
    cv2.imshow("Tracking", frame)

    # Qキーが押されたらループを終了します
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# リソースを解放します
cap.release()
cv2.destroyAllWindows()

### 【上級】OpenPoseを用いた骨格検出

このPythonスクリプトは、OpenCVとOpenPoseを使用して、リアルタイムの人体ポーズ推定を行います。主な機能は以下の通りです：

1. ウェブカメラからのビデオ入力を取得します。
2. 各フレームに対して、事前学習済みのOpenPoseモデル（COCOまたはMPI）を使用してポーズ推定を行います。
3. 検出された人体の各キーポイント（関節など）を画像上に表示します。
4. キーポイント間を線で結び、人体のスケルトンを描画します。
5. 結果をリアルタイムで表示し、キー入力があるまで処理を継続します。

In [6]:
# 必要なライブラリをインポート
import cv2  # OpenCVライブラリ：画像処理用
import time  # 時間計測用
import numpy as np  # 数値計算用

# ポーズ推定のモデルを選択（COCO or MPI）
MODE = "MPI"

# COCOモデルの設定
if MODE == "COCO":
    protoFile = "pose/coco/pose_deploy_linevec.prototxt"  # モデル構造ファイル
    weightsFile = "pose/coco/pose_iter_440000.caffemodel"  # 学習済みの重みファイル
    nPoints = 18  # キーポイントの数
    # キーポイント間の接続を定義（スケルトンの描画に使用）
    POSE_PAIRS = [ [1,0], [1,2], [1,5], [2,3], [3,4], [5,6],
                   [6,7], [1,8], [8,9], [9,10], [1,11], [11,12],
                   [12,13], [0,14], [0,15], [14,16], [15,17] ]

# MPIモデルの設定
elif MODE == "MPI":
    protoFile = "pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
    weightsFile = "pose/mpi/pose_iter_160000.caffemodel"
    nPoints = 15
    POSE_PAIRS = [ [0,1], [1,2], [2,3], [3,4], [1,5],
                   [5,6], [6,7], [1,14], [14,8], [8,9],
                   [9,10], [14,11], [11,12], [12,13] ]

# OpenPoseネットワークの読み込み
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

# ウェブカメラからの映像取得を初期化
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# 入力画像のサイズを設定
inWidth = 368
inHeight = 368

# キーポイント検出の閾値
threshold = 0.1

# メインループ：キー入力があるまで続く
while cv2.waitKey(1) < 0:
    t = time.time()  # フレーム処理時間の計測開始

    # カメラからフレームを読み込む
    hasFrame, frame = cap.read()
    if not hasFrame:
        break

    # フレームのサイズを取得
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    # フレームのコピーを作成（キーポイント描画用）
    frameCopy = np.copy(frame)

    # OpenPoseの入力形式に合わせて画像を前処理
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                    (0, 0, 0), swapRB=False, crop=False)

    # ネットワークに入力を設定
    net.setInput(inpBlob)

    # 推論を実行
    output = net.forward()

    # 出力のサイズを取得
    H = output.shape[2]
    W = output.shape[3]

    # キーポイントの検出
    points = []
    for i in range(nPoints):
        # 各キーポイントの確率マップを取得q
        probMap = output[0, i, :, :]
        
        # 確率マップから最大値とその位置を取得
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        
        # 画像上の座標に変換
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        # 閾値以上の確率を持つキーポイントを描画
        if prob > threshold:
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255),
                       thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2,
                        lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
        else:
            points.append(None)

    # スケルトンの描画
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]
        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB],
                     (0, 255, 255), 3)

    # 結果の表示
    cv2.imshow('Output-Keypoints', frameCopy)
    cv2.imshow('Output-Skeleton', frame)

# リソースの解放
cap.release()
cv2.destroyAllWindows()